In [2]:
import os
import numpy as np
from tqdm import tqdm

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR

import torchio as tio

from utils.utils import train, validate
from utils.vis import plot_mri
from utils.metric import dice_score_3d
from utils.dataset import BrainMRIDataset
from utils.loss import DiceCrossEntropyLoss
from models.Unet import UNet3D

## Constants

In [6]:
ROOT_DIR = './Data/'
BATCH_SIZE = 2
EPOCHS = 300
NUM_CLASSES = 4
DEVICE = 'mps' if torch.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

## Transforms

In [4]:
train_transform = transforms.Compose([
    transforms.ToTensor()
])

test_transform = transforms.Compose([
    transforms.ToTensor()
])

In [7]:
# TorchIO transformations for augmentation
train_transform = tio.Compose([
    tio.RandomAffine(scales=(0.9, 1.1), degrees=(10, 10, 10), translation=(5, 5, 5)),
    tio.RandomElasticDeformation(num_control_points=(7, 7, 7), max_displacement=(4, 4, 4)),
    tio.RandomFlip(axes=(0, 1, 2)),
    tio.RandomBiasField(coefficients=(0.1, 0.3)),
    tio.RescaleIntensity((0, 1))  # Normalize intensity to [0, 1]
])

val_transform = tio.Compose([
    tio.RescaleIntensity((0, 1))  # Only normalize intensity for validation
])

In [16]:
train_dataset = BrainMRIDataset(os.path.join(ROOT_DIR, 'train'), transform=None)
val_dataset = BrainMRIDataset(os.path.join(ROOT_DIR, 'val'), transform=None)

In [18]:
# Apply transforms to datasets
train_dataset = tio.SubjectsDataset(
    [tio.Subject(image=tio.ScalarImage(sample['image']),
                 mask=tio.LabelMap(sample['mask'])) for sample in train_dataset],
    transform=train_transform
)

val_dataset = tio.SubjectsDataset(
    [tio.Subject(image=tio.ScalarImage(tensor=sample['image']),
                 mask=tio.LabelMap(tensor=sample['mask'])) for sample in val_dataset],
    transform=val_transform
)

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (135366825.py, line 9)

## Models

In [6]:
model = UNet3D(in_channels=1, out_channels=NUM_CLASSES)
model = model.to(DEVICE)

## Loss & Optimizer

In [7]:
criterion = DiceCrossEntropyLoss(is_3d=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = CosineAnnealingLR(optimizer, T_max=10, eta_min=1e-5)

## Training

In [8]:
# Example training loop
for epoch in range(EPOCHS):
    model.train()
    epoch_loss = 0
    epoch_dice = 0
    progress_bar = tqdm(train_loader, total=len(train_loader), desc=f"Epoch {epoch + 1}/{EPOCHS}")
    for batch in progress_bar:
        images, masks = batch["image"].to(DEVICE), batch["mask"].to(DEVICE)  # Adjust keys if necessary

        # Forward pass
        outputs = model(images)

        # Compute loss
        loss = criterion(outputs, masks)
        epoch_loss += loss.item()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Dice score
        dice = dice_score_3d(outputs, masks, NUM_CLASSES)
        
        #update the progress bar
        progress_bar.set_postfix({"Loss": loss.item() / len(batch), "Avg Dice": np.mean(list(dice.values())), "WM Dice": dice[1], "GM Dice": dice[2], "CSF Dice": dice[3]})

    print(f"Epoch {epoch + 1}, Loss: {epoch_loss/len(train_loader):.4f}")
    print(f"Epoch {epoch + 1}, Dice: {np.mean(list(dice.values())):.4f}", f"WM Dice: {dice[1]:.4f}", f"GM Dice: {dice[2]:.4f}", f"CSF Dice: {dice[3]:.4f}")

Epoch 1/300: 100%|██████████| 5/5 [00:13<00:00,  2.64s/it, Loss=0.184, Avg Dice=0.498, WM Dice=7.2e-11, GM Dice=0.432, CSF Dice=0.57]     


Epoch 1, Loss: 0.6666
Epoch 1, Dice: 0.4980 WM Dice: 0.0000 GM Dice: 0.4318 CSF Dice: 0.5703


Epoch 2/300: 100%|██████████| 5/5 [00:11<00:00,  2.39s/it, Loss=0.181, Avg Dice=0.49, WM Dice=6.35e-11, GM Dice=0.479, CSF Dice=0.495]    


Epoch 2, Loss: 0.3398
Epoch 2, Dice: 0.4902 WM Dice: 0.0000 GM Dice: 0.4792 CSF Dice: 0.4954


Epoch 3/300: 100%|██████████| 5/5 [00:11<00:00,  2.38s/it, Loss=0.15, Avg Dice=0.588, WM Dice=5.81e-11, GM Dice=0.744, CSF Dice=0.614]    


...

Epoch 298, Loss: 0.0573
Epoch 298, Dice: 0.9345 WM Dice: 0.8792 GM Dice: 0.9378 CSF Dice: 0.9243


Epoch 299/300: 100%|██████████| 5/5 [00:12<00:00,  2.42s/it, Loss=0.0276, Avg Dice=0.931, WM Dice=0.923, GM Dice=0.905, CSF Dice=0.9]  


Epoch 299, Loss: 0.0583
Epoch 299, Dice: 0.9312 WM Dice: 0.9227 GM Dice: 0.9050 CSF Dice: 0.9000


Epoch 300/300: 100%|██████████| 5/5 [00:12<00:00,  2.44s/it, Loss=0.0245, Avg Dice=0.942, WM Dice=0.906, GM Dice=0.935, CSF Dice=0.929]

Epoch 300, Loss: 0.0554
Epoch 300, Dice: 0.9419 WM Dice: 0.9060 GM Dice: 0.9354 CSF Dice: 0.9290
